In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import brentq, minimize_scalar
import time
import warnings
import QuantLib as ql
import math
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
from scipy.special import erfc
import warnings
import time
import math


# Suppress warnings
warnings.filterwarnings('ignore')

# The risk-free rate is now a parameter that can be adjusted for each method
# Default value - will be overridden by parameter versions
DEFAULT_R = 0.03


# 1. Black-Scholes European approximation with parameters
def compute_iv_euro(row, r=DEFAULT_R, max_vol=3.0, min_vol=0.001, max_iter=100):
    try:
        S = row['current_underlying_mean_mid']  # Underlying price
        K = row['strike']  # Strike price
        T = row['maturity'] / 365  # Time to expiry in years
        price = row['mean_mid']  # Option price
        is_call = row['is_call'] == 1  # Option type (1=call, 0=put)

        # Calculate Black-Scholes implied volatility
        def objective(sigma):
            return black_scholes_price(S, K, T, r, sigma, is_call) - price

        # Use brentq to find the implied volatility with customizable parameters
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv
    except:
        return np.nan


# Black-Scholes pricing formula
def black_scholes_price(S, K, T, r, sigma, is_call):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    from scipy.stats import norm
    if is_call:
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


# 2. Binomial Tree method with parameters
def implied_vol_binomial(row, r=DEFAULT_R, steps=100, max_vol=3.0, min_vol=0.001, max_iter=100):
    try:
        S = row['current_underlying_mean_mid']
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        def objective(sigma):
            model_price = binomial_tree_price(S, K, T, r, sigma, is_call, steps=steps)
            return model_price - price

        # Find the implied volatility using brentq with customizable parameters
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv
    except:
        return np.nan


# Binomial Tree pricing model with parameters
def binomial_tree_price(S, K, T, r, sigma, is_call, steps=100):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    prices = np.zeros(steps + 1)
    for i in range(steps + 1):
        prices[i] = S * (u ** (steps - i)) * (d ** i)

    # Initialize option values at maturity
    if is_call:
        values = np.maximum(0, prices - K)
    else:
        values = np.maximum(0, K - prices)

    # Work backwards through the tree
    for step in range(steps - 1, -1, -1):
        for i in range(step + 1):
            # Calculate asset price at this node
            asset_price = S * (u ** (step - i)) * (d ** i)

            # Calculate option value from children nodes
            option_value = np.exp(-r * dt) * (p * values[i] + (1 - p) * values[i + 1])

            # Check for early exercise
            if is_call:
                exercise_value = max(0, asset_price - K)
            else:
                exercise_value = max(0, K - asset_price)

            # Take maximum of continuation and exercise
            values[i] = max(option_value, exercise_value)

    # Return the option price (root of the tree)
    return values[0]


import QuantLib as ql


def quantlib_binomial_spot_iv(row, r=0.03, steps=200, max_vol=3.0, min_vol=0.001, max_iter=100):
    #100 steps is not enough to get normal answer
    try:
        # Extract option parameters
        S = row['current_underlying_mean_mid']  # Use spot price instead of futures price
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        # Setup dates
        calculation_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = calculation_date
        maturity_date = calculation_date + ql.Period(int(T * 365), ql.Days)

        def objective(sigma):
            try:
                # Create the spot price handle
                spot_handle = ql.QuoteHandle(ql.SimpleQuote(S))

                # Create yield curves for regular American options
                risk_free_ts = ql.YieldTermStructureHandle(
                    ql.FlatForward(calculation_date, r, ql.Actual365Fixed())
                )
                dividend_ts = ql.YieldTermStructureHandle(
                    ql.FlatForward(calculation_date, 0.0, ql.Actual365Fixed())  # No dividend
                )

                # Create volatility surface
                vol_ts = ql.BlackVolTermStructureHandle(
                    ql.BlackConstantVol(calculation_date, ql.NullCalendar(), sigma, ql.Actual365Fixed())
                )

                # Create Black-Scholes process for standard options
                process = ql.BlackScholesMertonProcess(
                    spot_handle,  # Spot price
                    dividend_ts,  # Dividend yield
                    risk_free_ts,  # Risk-free rate
                    vol_ts  # Volatility
                )

                # Create binomial engine
                engine = ql.BinomialVanillaEngine(process, "crr", steps)

                # Create the option
                option_type = ql.Option.Call if is_call else ql.Option.Put
                payoff = ql.PlainVanillaPayoff(option_type, K)
                exercise = ql.AmericanExercise(calculation_date, maturity_date)
                option = ql.VanillaOption(payoff, exercise)

                # Price the option
                option.setPricingEngine(engine)
                model_price = option.NPV()

                return model_price - price

            except Exception as e:
                print(f"Error in objective: {e}")
                return 0

        # Find the implied volatility
        from scipy.optimize import brentq
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv

    except Exception as e:
        print(f"Error: {e}")
        return np.nan


# 3. Trinomial Tree method with parameters
def implied_vol_trinomial(row, r=DEFAULT_R, steps=100, max_vol=3.0, min_vol=0.001, max_iter=100):
    try:
        S = row['current_underlying_mean_mid']
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        def objective(sigma):
            model_price = trinomial_tree_price(S, K, T, r, sigma, is_call, steps=steps)
            return model_price - price

        # Find the implied volatility using brentq with customizable parameters
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv
    except Exception as e:
        print(f"Error: {e}")
        return np.nan


# Trinomial Tree pricing model with parameters
def trinomial_tree_price(S, K, T, r, sigma, is_call, steps=50):
    dt = T / steps

    # Trinomial tree parameters
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = 1 / u
    m = 1.0

    # Risk-neutral probabilities
    pu = ((np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) /
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))) ** 2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) /
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))) ** 2
    pm = 1 - pu - pd

    # Initialize arrays with proper size
    max_nodes = 2 * steps + 1
    price_tree = np.zeros((steps + 1, max_nodes))
    option_tree = np.zeros((steps + 1, max_nodes))

    # Build the tree using centered indexing
    for i in range(steps + 1):  # Time steps
        for j in range(-i, i + 1):  # Node positions at this time step
            idx = j + steps  # Convert to array index (center is at position 'steps')
            price_tree[i, idx] = S * (u ** j)

    # Initialize option values at maturity
    for j in range(-steps, steps + 1):
        idx = j + steps
        if is_call:
            option_tree[steps, idx] = max(0, price_tree[steps, idx] - K)
        else:
            option_tree[steps, idx] = max(0, K - price_tree[steps, idx])

    # Work backwards through the tree
    for i in range(steps - 1, -1, -1):
        for j in range(-i, i + 1):
            idx = j + steps

            # Get indices for the three children nodes
            idx_up = (j + 1) + steps
            idx_mid = j + steps
            idx_down = (j - 1) + steps

            # Calculate expected option value
            expected = (pu * option_tree[i + 1, idx_up] +
                        pm * option_tree[i + 1, idx_mid] +
                        pd * option_tree[i + 1, idx_down]) * np.exp(-r * dt)

            # Calculate exercise value
            if is_call:
                exercise = max(0, price_tree[i, idx] - K)
            else:
                exercise = max(0, K - price_tree[i, idx])

            # Take maximum of continuation and exercise
            option_tree[i, idx] = max(expected, exercise)

    # Return option price at root (time 0, middle node)
    return option_tree[0, steps]

# 4. Barone-Adesi and Whaley (BAW) approximation with parameters
def baw_american_iv(row, r=DEFAULT_R, q=0.0, max_vol=3.0, min_vol=0.001, max_iter=100):
    try:
        S = row['current_underlying_mean_mid']
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        def objective(sigma):
            model_price = american_option_baw(S, K, T, r, sigma, is_call, q=q)
            return model_price - price

        # Find the implied volatility with customizable parameters
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv
    except:
        return np.nan

# using quantlib
def quantlib_trinomial_iv(row, r=0.03, steps=100, max_vol=3.0, min_vol=0.001, max_iter=100):
    """Calculate implied volatility using QuantLib's trinomial tree method"""
    try:
        # Extract option parameters
        S = row['current_underlying_mean_mid']  # Spot price
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        # Setup dates
        calculation_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = calculation_date
        maturity_date = calculation_date + ql.Period(int(T * 365), ql.Days)

        def objective(sigma):
            try:
                # Create the spot price handle
                spot_handle = ql.QuoteHandle(ql.SimpleQuote(S))

                # Create yield curves
                risk_free_ts = ql.YieldTermStructureHandle(
                    ql.FlatForward(calculation_date, r, ql.Actual365Fixed())
                )
                dividend_ts = ql.YieldTermStructureHandle(
                    ql.FlatForward(calculation_date, 0.0, ql.Actual365Fixed())  # No dividend
                )

                # Create volatility surface
                vol_ts = ql.BlackVolTermStructureHandle(
                    ql.BlackConstantVol(calculation_date, ql.NullCalendar(), sigma, ql.Actual365Fixed())
                )

                # Create the process
                process = ql.BlackScholesMertonProcess(
                    spot_handle, dividend_ts, risk_free_ts, vol_ts
                )

                # Create trinomial engine - use TrinomialVanillaEngine
                engine = ql.TrinomialVanillaEngine(process, steps)

                # Create the option
                option_type = ql.Option.Call if is_call else ql.Option.Put
                payoff = ql.PlainVanillaPayoff(option_type, K)
                exercise = ql.AmericanExercise(calculation_date, maturity_date)
                option = ql.VanillaOption(payoff, exercise)

                # Price the option
                option.setPricingEngine(engine)
                model_price = option.NPV()

                return model_price - price

            except Exception as e:
                print(f"Error in objective: {e}")
                return 0

        # Find the implied volatility
        from scipy.optimize import brentq
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv

    except Exception as e:
        print(f"Error: {e}")
        return np.nan


def american_option_baw(S, K, T, r, sigma, is_call, q=0.0):
    """Barone-Adesi and Whaley approximation for American options"""
    from scipy.stats import norm

    # First calculate European option price
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if is_call:
        euro_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        euro_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

    # Early exercise premium
    if is_call:
        if r <= q:  # No early exercise benefit
            return euro_price
        else:
            # Call option approximation
            b = r - q
            M = 2 * r / (sigma ** 2)
            N = 2 * b / (sigma ** 2)
            K_inf = K * N / (N - 1)
            q2 = (-(N - 1) + np.sqrt((N - 1) ** 2 + 4 * M / K)) / 2
            a2 = (K / q2) * (1 - np.exp(-r * T) * norm.cdf(d2))

            if S >= K_inf:
                return S - K
            else:
                return euro_price + a2 * (S / K_inf) ** q2
    else:  # Put option
        if r >= q:  # Early exercise calculation
            b = r - q
            M = 2 * r / (sigma ** 2)
            N = 2 * b / (sigma ** 2)
            K_0 = K * N / (N + 1)
            q1 = (-(N - 1) - np.sqrt((N - 1) ** 2 + 4 * M / K)) / 2
            a1 = -(K / q1) * (1 - np.exp(-r * T) * norm.cdf(-d2))

            if S <= K_0:
                return K - S
            else:
                return euro_price + a1 * (S / K_0) ** q1
        else:
            return euro_price


# 5. Bjerksund-Stensland approximation with parameters
def bjerksund_iv(row, r=DEFAULT_R, q=0.0, max_vol=3.0, min_vol=0.001, max_iter=100):
    try:
        S = row['current_underlying_mean_mid']
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        def objective(sigma):
            model_price = american_option_bjerksund(S, K, T, r, sigma, is_call, q=q)
            return model_price - price

        # Find the implied volatility with customizable parameters
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv
    except:
        return np.nan


def american_option_bjerksund(S, K, T, r, sigma, is_call, q=0.0):
    """Simplified Bjerksund-Stensland approximation for American options"""
    from scipy.stats import norm

    # For put options, use put-call transformation
    if not is_call:
        # P(S,K,T,r,σ,q) = C(K,S,T,q,σ,r)
        return american_option_bjerksund(K, S, T, q, sigma, True, r)

    # Check boundary conditions
    if T <= 0:
        return max(0, S - K)

    # Calculate European price (used as fallback)
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    euro_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    b = r - q

    # Check for early exercise
    if b >= r:  # No early exercise for this case
        return euro_price

    # Calculate parameters for the approximation
    beta = (0.5 - b / (sigma ** 2)) + np.sqrt((b / (sigma ** 2) - 0.5) ** 2 + 2 * r / (sigma ** 2))
    B_infty = beta * K / (beta - 1)
    B_0 = max(K, r * K / b)

    h1 = -(b * T + 2 * sigma * np.sqrt(T)) * K / (B_0 - K)
    h2 = -(b * T - 2 * sigma * np.sqrt(T)) * K / (B_0 - K)

    I1 = B_0 - (B_0 - K) * np.exp(h1)
    I2 = B_0 - (B_0 - K) * np.exp(h2)

    alpha = (I1 - I2) / (B_infty - B_0)

    # Exercise boundary
    B = alpha * B_infty + (1 - alpha) * B_0

    # If S ≥ B, immediate exercise
    if S >= B:
        return S - K

    # Otherwise use approximation formula
    lambda_val = -2 * b / (sigma ** 2)

    d1 = (np.log(S / B) + (b + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    part1 = (S * np.exp((b - r) * T) * norm.cdf(d1)) - (K * np.exp(-r * T) * norm.cdf(d1 - sigma * np.sqrt(T)))
    part2 = (S * np.exp((b - r) * T) * norm.cdf(-d1)) - (K * np.exp(-r * T) * norm.cdf(-d1 + sigma * np.sqrt(T)))

    return part1 - part2 * (S / B) ** lambda_val


# 6. Monte Carlo simulation with LSM with parameters
def monte_carlo_iv(row, r=DEFAULT_R, M=1000, N=50, max_vol=3.0, min_vol=0.001, max_iter=50, seed=42):
    try:
        S = row['current_underlying_mean_mid']
        K = row['strike']
        T = row['maturity'] / 365
        price = row['mean_mid']
        is_call = row['is_call'] == 1

        # For efficiency, skip very deep ITM options
        intrinsic = max(0, S - K) if is_call else max(0, K - S)
        if intrinsic / price > 0.95:
            return np.nan

        def objective(sigma):
            model_price = lsm_american_option_price(S, K, T, r, sigma, is_call, M=M, N=N,seed=seed)
            return model_price - price

        # Find the implied volatility with customizable parameters
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter)
        return iv
    except:
        return np.nan


def simulate_paths(S, r, sigma, T, M, N, seed=42):
    """Simulate asset price paths with optional seed parameter"""
    dt = T / N
    paths = np.zeros((M, N + 1))
    paths[:, 0] = S

    # Generate random paths with optional seed for reproducibility
    np.random.seed(seed)
    Z = np.random.standard_normal((M, N))
    for t in range(1, N + 1):
        paths[:, t] = paths[:, t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z[:, t - 1])

    return paths


def lsm_american_option_price(S, K, T, r, sigma, is_call, M=1000, N=50, seed=42, min_itm_paths=5):
    """Price American option using Least Squares Monte Carlo method with parameters"""
    # Simulate paths
    paths = simulate_paths(S, r, sigma, T, M, N, seed=seed)
    dt = T / N

    # Initialize payoffs at maturity
    if is_call:
        payoffs = np.maximum(0, paths[:, -1] - K)
    else:
        payoffs = np.maximum(0, K - paths[:, -1])

    # Working backwards through time
    for t in range(N - 1, 0, -1):
        # Identify in-the-money paths
        if is_call:
            itm = paths[:, t] > K
        else:
            itm = paths[:, t] < K

        if sum(itm) > min_itm_paths:  # Need enough ITM paths for regression (now parameterized)
            # Current stock prices for ITM options
            S_itm = paths[itm, t]
            # Future discounted cashflows for ITM options
            V_itm = payoffs[itm] * np.exp(-r * dt * (N - t))
            # Regression basis functions (polynomial)
            X = np.column_stack([np.ones(len(S_itm)), S_itm, S_itm ** 2])
            # Fit regression model
            beta, _, _, _ = np.linalg.lstsq(X, V_itm, rcond=None)
            # Expected continuation values
            C = np.dot(X, beta)
            # Immediate exercise values
            if is_call:
                exercise = np.maximum(0, S_itm - K)
            else:
                exercise = np.maximum(0, K - S_itm)

            # Exercise decision
            ex_idx = exercise > C

            # Update payoffs
            payoffs_new = np.copy(payoffs)
            ex_path_idx = np.where(itm)[0][ex_idx]
            payoffs_new[ex_path_idx] = exercise[ex_idx] * np.exp(-r * dt * t)

            # Only consider unexercised paths for future iterations
            payoffs = payoffs_new

    # Option price is the average of all discounted payoffs
    return np.mean(payoffs)

def european_put_price(S, K, r, sigma, T):
    """Calculate European put price"""
    if T <= 0 or sigma <= 0:
        return max(K - S, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


def zhu_simplified_american_put_price(S, K, r, sigma, T, n_terms=15):
    """
    Calculate American put price using simplified Zhu formula
    For OTM puts, American = European approximately
    """

    # ITM put: Apply early exercise premium
    european_price = european_put_price(S, K, r, sigma, T)

    # Early exercise premium (simplified approximation)
    # In practice, this should use the full Zhu series expansion
    gamma = 2 * r / (sigma ** 2)
    tau = T * sigma ** 2 / 2

    # Approximate early exercise premium
    optimal_boundary_factor = np.exp(-gamma * tau)
    early_premium = european_price * (1 - optimal_boundary_factor)

    return european_price + early_premium


def implied_vol_zhu_simplified(row, r=0.03, min_vol=0.001, max_vol=2.0, max_iter=100, debug=False):
    """Calculate implied volatility using Zhu's formula for American puts"""
    S = row['current_underlying_mean_mid']
    K = row['strike']
    T = row['maturity'] / 365
    price = row['mean_mid']
    is_call = row['is_call'] == 1

    # Skip call options or invalid data
    if is_call or not all(np.isfinite([S, K, T, price])) or any(x <= 0 for x in [S, K, T, price]):
        return np.nan

    intrinsic = max(0, K - S)

    # For OTM options, price should be less than strike
    if S >= K and price > K:
        print('impossible price')
        return np.nan

    # Price should be > intrinsic value
    if price <= intrinsic * 1.001:
        print('all most all intrinsic value')
        return np.nan

    def objective(sigma):
        if sigma <= 1e-6 or sigma >= 10.0:
            return float('inf')
        try:
            model_price = zhu_simplified_american_put_price(S, K, r, sigma, T)
            if not np.isfinite(model_price):
                return float('inf')
            error = model_price - price

            if debug:
                euro_price = european_put_price(S, K, r, sigma, T)
                print(f"  sigma={sigma:.6f}: model={model_price:.6f}, euro={euro_price:.6f}, error={error:.6f}")

            return error
        except Exception as e:
            print(f"  Error with sigma={sigma}: {str(e)}")
            return float('inf')

    # First check if solutions exist at boundaries
    f_min = objective(min_vol)
    f_max = objective(max_vol)

    if debug:
        print(f"\nIV calculation:")
        print(f"  S={S:.4f}, K={K:.4f}, T={T:.6f}, price={price:.6f}")
        print(f"  Intrinsic={intrinsic:.6f}")
        print(f"  f(min_vol)={f_min:.6f}, f(max_vol)={f_max:.6f}")

    if not np.isfinite(f_min) or not np.isfinite(f_max) or f_min * f_max >= 0:
        if debug:
            print(f"  No bracketing solution found")
        return np.nan

    try:
        iv = brentq(objective, min_vol, max_vol, maxiter=max_iter, xtol=1e-8)


        return iv
    except Exception as e:
        print(f"  Exception: {str(e)}")
        return np.nan

In [2]:
# Load the data
file_path = 'sample_data.csv'  # Use your file path
df = pd.read_csv(file_path)
df = df[df['is_call'] == 0]
df=df.reset_index(drop=True)

# Ensure time_to_expiry column exists
if 'time_to_expiry' not in df.columns:
    df['time_to_expiry'] = df['maturity'] / 365.0

# discount back to current price
df['current_underlying_mean_mid'] = df['underly_mean_mid'] * np.exp(-DEFAULT_R * df['time_to_expiry'])
# Use a subset of the data for faster testing
sample_size = 10
df_filtered = df[:sample_size]

print(f"Processing {sample_size} option records...")

Processing 10 option records...


In [3]:
# 1. Binomial Tree method with different parameters
t0 = time.time()

# Run with default parameters
df_filtered['iv_binomial_default'] = df_filtered.apply(implied_vol_binomial, axis=1)

# Run with custom parameters (example: more steps for better accuracy)
# df_filtered['iv_binomial_middle_steps'] = df_filtered.apply(
#     lambda row: implied_vol_binomial(row, steps=200), axis=1)

# df_filtered['iv_binomial_high_steps'] = df_filtered.apply(
#     lambda row: implied_vol_binomial(row, steps=500), axis=1)



# Run with different risk-free rate
# df_filtered['iv_binomial_higher_r'] = df_filtered.apply(
#     lambda row: implied_vol_binomial(row, r=0.04), axis=1)

t1 = time.time()
print(f'Time spent for binomial methods: {t1 - t0:.2f} seconds')
print(f'Default valid results: {df_filtered["iv_binomial_default"].notna().sum()} out of {sample_size}')
# print(f'High steps valid results: {df_filtered["iv_binomial_high_steps"].notna().sum()} out of {sample_size}')
# print(f'Higher r valid results: {df_filtered["iv_binomial_higher_r"].notna().sum()} out of {sample_size}')

Time spent for binomial methods: 0.48 seconds
Default valid results: 10 out of 10


In [4]:
 # binomial tree using quantlib library
df_filtered['iv_benchmark'] = df_filtered.apply(
lambda row: quantlib_binomial_spot_iv(row, steps=5000), axis=1)

t2 = time.time()

df_filtered['iv_ql_binomial_default'] = df_filtered.apply(quantlib_binomial_spot_iv, axis=1)
# df_filtered['iv_ql_binomial_middle_steps'] = df_filtered.apply(
#     lambda row: quantlib_binomial_spot_iv(row, steps=300), axis=1)
# df_filtered['iv_ql_binomial_high_steps'] = df_filtered.apply(
#     lambda row: quantlib_binomial_spot_iv(row, steps=500), axis=1)
# df_filtered['iv_ql_binomial_highest_steps'] = df_filtered.apply(
#     lambda row: quantlib_binomial_spot_iv(row, steps=1000), axis=1)


# df_filtered['iv_ql_binomial_more_iters_steps'] = df_filtered.apply(
#     lambda row: quantlib_binomial_spot_iv(row, max_iter=200), axis=1)

t3 = time.time()
print(f'Time spent for quantlib binomial methods: {t3 - t2:.2f} seconds')
print(f'Default valid results: {df_filtered["iv_ql_binomial_default"].notna().sum()} out of {sample_size}')
print(f'benchmark valid results: {df_filtered["iv_benchmark"].notna().sum()} out of {sample_size}')
# print(f'High steps valid results: {df_filtered["iv_ql_binomial_high_steps"].notna().sum()} out of {sample_size}')
# print(f'Higher r iters results: {df_filtered["iv_ql_binomial_more_iters_steps"].notna().sum()} out of {sample_size}')

print()

Time spent for quantlib binomial methods: 0.02 seconds
Default valid results: 10 out of 10
benchmark valid results: 10 out of 10



In [5]:
# 2. Trinomial Tree method with different parameters
t4 = time.time()

# Run with default parameters
# df_filtered['iv_trinomial_default'] = df_filtered.apply(implied_vol_trinomial, axis=1)
df_filtered['iv_trinomial_default'] = df_filtered.apply(implied_vol_trinomial, axis=1)

# Run with more steps for better accuracy
# df_filtered['iv_trinomial_middle_steps'] = df_filtered.apply(
#     lambda row: implied_vol_trinomial(row, steps=300), axis=1)

# df_filtered['iv_trinomial_high_steps'] = df_filtered.apply(
#     lambda row: implied_vol_trinomial(row, steps=500), axis=1)


# df_filtered['iv_ql_trinomial_more_iters_steps'] = df_filtered.apply(
#     lambda row: implied_vol_trinomial(row, max_iter=200), axis=1)

t5 = time.time()
print(f'Time spent for Trinomial Tree methods: {t5 - t4:.2f} seconds')
print(f'Default valid results: {df_filtered["iv_trinomial_default"].notna().sum()} out of {sample_size}')
# print(f'Middle steps valid results: {df_filtered["iv_trinomial_middle_steps"].notna().sum()} out of {sample_size}')
# print(f'Most steps valid results: {df_filtered["iv_trinomial_high_steps"].notna().sum()} out of {sample_size}')
# print(f'Higher iters_steps valid results: {df_filtered["iv_ql_trinomial_more_iters_steps"].notna().sum()} out of {sample_size}')

    #trinomial tree does not have popular package can use

Time spent for Trinomial Tree methods: 1.12 seconds
Default valid results: 10 out of 10


In [6]:
# 3. Barone-Adesi and Whaley (BAW) method with different parameers
t6 = time.time()

# Run with default parameters
df_filtered['iv_baw_default'] = df_filtered.apply(baw_american_iv, axis=1)


# Run with higher max_iter
# df_filtered['iv_baw_higher_iter'] = df_filtered.apply(
#     lambda row: baw_american_iv(row, max_iter=200), axis=1)


# df_filtered['iv_baw_higher_r'] = df_filtered.apply(
#     lambda row: baw_american_iv(row,  r=0.04), axis=1)

# more iteration make almost no difference
# is more sensitive to r than iteration

t7 = time.time()
print(f'Time spent for BAW methods: {t7 - t6:.2f} seconds')
print(f'Default valid results: {df_filtered["iv_baw_default"].notna().sum()} out of {sample_size}')
# print(f'Higher iter valid results: {df_filtered["iv_baw_higher_iter"].notna().sum()} out of {sample_size}')


Time spent for BAW methods: 0.02 seconds
Default valid results: 10 out of 10


In [7]:
# 4. Bjerksund-Stensland method with different parameters
t8 = time.time()

# Run with default parameters
df_filtered['iv_bjerksund_default'] = df_filtered.apply(bjerksund_iv, axis=1)

# df_filtered['iv_bjerksund_more_iteration'] =  df_filtered.apply(
#     lambda row: bjerksund_iv(row, max_iter=500), axis=1)

# # Run with different min/max volatility bounds
# df_filtered['iv_bjerksund_wider_bounds'] = df_filtered.apply(
#     lambda row: bjerksund_iv(row, min_vol=0.0001, max_vol=5.0), axis=1)

# # Run with different min/max volatility bounds
# df_filtered['iv_bjerksund_smaller_bounds'] = df_filtered.apply(
#     lambda row: bjerksund_iv(row, min_vol=0.1, max_vol=0.5), axis=1)

# # Run with different risk-free rate and dividend yield
# df_filtered['iv_bjerksund_diff_r'] = df_filtered.apply(
#     lambda row: bjerksund_iv(row, r=0.02), axis=1)

t9 = time.time()
#put 都算不出来 算不出来的还是算不出出来，wider bound 和 more iteration基本没用，r影响比其他都大

print(f'Time spent for Bjerksund-Stensland methods: {t9 - t8:.2f} seconds')
print(f'Default valid results: {df_filtered["iv_bjerksund_default"].notna().sum()} out of {sample_size}')
# print(f'More Iteration valid results: {df_filtered["iv_bjerksund_more_iteration"].notna().sum()} out of {sample_size}')
# print(f'Wider bounds valid results: {df_filtered["iv_bjerksund_wider_bounds"].notna().sum()} out of {sample_size}')
# print(f'Smaller bounds valid results: {df_filtered["iv_bjerksund_smaller_bounds"].notna().sum()} out of {sample_size}')
# print(f'Different r valid results: {df_filtered["iv_bjerksund_diff_r"].notna().sum()} out of {sample_size}')


Time spent for Bjerksund-Stensland methods: 0.00 seconds
Default valid results: 0 out of 10


In [8]:
# 5. Black-Scholes European approximation with different parameters
t10 = time.time()

# Run with default parameters
df_filtered['iv_euro_default'] = df_filtered.apply(compute_iv_euro, axis=1)

# # Run with higher risk-free rate
# df_filtered['iv_euro_higher_r'] = df_filtered.apply(
#     lambda row: compute_iv_euro(row, r=0.04), axis=1)

# # Run with different max iterations
# df_filtered['iv_euro_lower_iter'] = df_filtered.apply(
#     lambda row: compute_iv_euro(row, max_iter=50), axis=1)

# # Run with different max iterations
# df_filtered['iv_euro_higher_iter'] = df_filtered.apply(
#     lambda row: compute_iv_euro(row, max_iter=300), axis=1)

t11 = time.time()
print(f'Time spent for European approximation methods: {t11 - t10:.2f} seconds')
print(f'Default valid results: {df_filtered["iv_euro_default"].notna().sum()} out of {sample_size}')
# print(f'Higher r valid results: {df_filtered["iv_euro_higher_r"].notna().sum()} out of {sample_size}')
# print(f'Lower iter valid results: {df_filtered["iv_euro_lower_iter"].notna().sum()} out of {sample_size}')
# print(f'Higher iter valid results: {df_filtered["iv_euro_higher_iter"].notna().sum()} out of {sample_size}')


Time spent for European approximation methods: 0.01 seconds
Default valid results: 10 out of 10


In [9]:
# 6. Monte Carlo method with different parameters (more time-consuming)
t12 = time.time()

df_filtered['iv_monte_carlo_default'] = df_filtered.apply(monte_carlo_iv, axis=1)

# Run with different seed for different random numbers
# df_filtered['iv_monte_carlo_diff_seed'] = df_filtered.apply(
#     lambda row: monte_carlo_iv(row, seed=123), axis=1)

t13 = time.time()
print(f'Time spent for Monte Carlo methods: {t13 - t12:.2f} seconds')


# t14 = time.time()

# Run with more paths for better accuracy (but slower)
# df_filtered['iv_monte_carlo_more_paths'] = df_filtered.apply(
#     lambda row: monte_carlo_iv(row, M=2000, N=100), axis=1)

# t15 = time.time()
# print(f'Time spent for Monte Carlo methods: {t15 - t14:.2f} seconds')

print(
    f'Default valid results: {df_filtered["iv_monte_carlo_default"].notna().sum()} out of {len(df_filtered)}')
# print(
#     f'More paths valid results: {df_filtered["iv_monte_carlo_more_paths"].notna().sum()} out of {len(df_filtered)}')
# print(
#     f'Different seed valid results: {df_filtered["iv_monte_carlo_diff_seed"].notna().sum()} out of {len(df_filtered)}')



Time spent for Monte Carlo methods: 1.10 seconds
Default valid results: 10 out of 10


In [10]:
# t16 = time.time()
# # Run with more paths for best accuracy (but slower)
# df_filtered['iv_monte_carlo_most_paths'] = df_filtered.apply(
#     lambda row: monte_carlo_iv(row, M=4000, N=200), axis=1)

# df_filtered['iv_monte_carlo_most_paths_other_seed'] = df_filtered.apply(
#     lambda row: monte_carlo_iv(row, M=4000, N=200, seed=123), axis=1)

# t17 = time.time()
# print(f'Time spent for Monte Carlo methods: {t17 - t16:.2f} seconds')


In [11]:
# 7. simplified zhu's method

t18 = time.time()

df_filtered['iv_zhu'] = df_filtered.apply(
    lambda row: implied_vol_zhu_simplified(row), axis=1)

t19 = time.time()

print(f'Time spent for zhu methods: {t19 - t18:.2f} seconds')

print()


Time spent for zhu methods: 0.01 seconds



In [12]:
# 7. simplified zhu's method

t18 = time.time()

df_filtered['iv_zhu'] = df_filtered.apply(
    lambda row: implied_vol_zhu_simplified(row), axis=1)

t19 = time.time()

print(f'Time spent for zhu methods: {t19 - t18:.2f} seconds')

Time spent for zhu methods: 0.01 seconds


In [13]:
df_filtered

,Unnamed: 0,minute_str,Instrument,mean_ask,mean_bid,cumcashvol,cumvolume,openinterest,is_call,strike,...,current_underlying_mean_mid,iv_binomial_default,iv_benchmark,iv_ql_binomial_default,iv_trinomial_default,iv_baw_default,iv_bjerksund_default,iv_euro_default,iv_monte_carlo_default,iv_zhu
0,0,2024-06-03 21:00,CF2409P13800,84.250000,79.625000,5295.0,13.0,3955.0,0.0,13800.0,...,14889.711256,0.175899,0.175654,0.175845,0.175862,0.056804,NaN,0.176227,0.168773,0.175898
1,1,2024-06-03 21:00,CF2409P14200,142.631579,138.578947,12745.0,18.0,0.0,0.0,14200.0,...,14896.285167,0.164526,0.164773,0.164635,0.164654,0.057061,NaN,0.165688,0.160572,0.165271
2,2,2024-06-03 21:00,CF2409P14400,189.666667,184.888889,7500.0,8.0,5919.0,0.0,14400.0,...,14890.470763,0.159980,0.160323,0.160155,0.160175,0.057575,NaN,0.161516,0.157307,0.161027
3,3,2024-06-03 21:00,CF2409P14600,252.000000,250.500000,25100.0,20.0,7909.0,0.0,14600.0,...,14886.604184,0.157727,0.158099,0.158210,0.158236,0.058950,NaN,0.159709,0.156263,0.159113
4,5,2024-06-03 21:00,CF2409P14800,326.272727,320.272727,21100.0,13.0,0.0,0.0,14800.0,...,14888.411935,0.153680,0.154067,0.153879,0.153907,0.060766,NaN,0.156271,0.153403,0.155546
5,7,2024-06-03 21:00,CF2409P15000,421.071429,406.928571,2000.0,1.0,0.0,0.0,15000.0,...,14891.575499,0.151237,0.151619,0.151531,0.151561,0.064132,NaN,0.154704,0.147725,0.153796
6,12,2024-06-03 21:01,CF2409P13400,50.000000,48.733333,7695.0,31.0,7920.0,0.0,13400.0,...,14891.244078,0.189580,0.189736,0.189979,0.189995,0.057194,NaN,0.190132,0.180348,0.189852
7,13,2024-06-03 21:01,CF2409P13800,83.032258,81.000000,5705.0,14.0,3956.0,0.0,13800.0,...,14891.655682,0.176145,0.175896,0.176090,0.176108,0.056840,NaN,0.176468,0.169006,0.176139
8,14,2024-06-03 21:01,CF2409P14000,107.161290,105.354839,27030.0,51.0,10167.0,0.0,14000.0,...,14891.575499,0.169299,0.169245,0.169292,0.169311,0.056737,NaN,0.169967,0.162856,0.169603
9,15,2024-06-03 21:01,CF2409P14200,144.983607,141.491803,16370.0,23.0,11447.0,0.0,14200.0,...,14891.371757,0.165371,0.165573,0.165402,0.165421,0.057246,NaN,0.166490,0.160978,0.166068


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

def analyze_iv_results(df_filtered, benchmark_col='iv_benchmark'):
    """
    Calculate error metrics for each IV method compared to the benchmark.
    Treats NaN values as zeros for error calculation.
    
    Args:
        df_filtered: DataFrame containing IV calculation results
        benchmark_col: Column name for the benchmark IV values
        
    Returns:
        DataFrame with error metrics
    """
    # Get all IV columns except benchmark
    iv_cols = [col for col in df_filtered.columns if col.startswith('iv_') and col != benchmark_col]
    
    # Create results dataframe
    results = pd.DataFrame(index=iv_cols)
    
    # Calculate error metrics for each method
    for col in iv_cols:
        # For rows where benchmark has valid values
        benchmark_valid = df_filtered[benchmark_col].notna()
        
        if benchmark_valid.sum() > 0:
            # Create a copy of the data with NaN values filled with zeros
            filled_df = df_filtered.copy()
            filled_df[col].fillna(0, inplace=True)
            filled_df[benchmark_col].fillna(0, inplace=True)
            
            # Calculate errors where benchmark is valid
            errors = filled_df.loc[benchmark_valid, col] - filled_df.loc[benchmark_valid, benchmark_col]
            abs_errors = np.abs(errors)
            # Avoid division by zero in percentage calculation
            percent_errors = abs_errors / np.maximum(filled_df.loc[benchmark_valid, benchmark_col], 1e-10) * 100
            
            # Calculate how many values were actually valid in the method
            valid_count = df_filtered.loc[benchmark_valid, col].notna().sum()
            
            # Store metrics
            results.loc[col, 'Total Count'] = benchmark_valid.sum()
            results.loc[col, 'Valid Count'] = valid_count
            results.loc[col, 'Valid Percentage'] = valid_count / benchmark_valid.sum() * 100
            results.loc[col, 'Mean Absolute Error'] = abs_errors.mean()
            results.loc[col, 'RMSE'] = np.sqrt((errors**2).mean())
            results.loc[col, 'Mean % Error'] = percent_errors.mean()
            results.loc[col, 'Max Absolute Error'] = abs_errors.max()
        else:
            # No valid benchmark values
            results.loc[col, 'Total Count'] = 0
            results.loc[col, 'Valid Count'] = 0
            results.loc[col, 'Valid Percentage'] = 0
            results.loc[col, 'Mean Absolute Error'] = np.nan
            results.loc[col, 'RMSE'] = np.nan
            results.loc[col, 'Mean % Error'] = np.nan
            results.loc[col, 'Max Absolute Error'] = np.nan
    
    # Sort by RMSE (lower is better)
    return results.sort_values('RMSE')

def plot_iv_comparison(df_filtered, benchmark_col='iv_benchmark', output_file='iv_comparison.png'):
    """
    Create scatter plots comparing each method to the benchmark.
    Fills NaN values with zeros for visualization.
    
    Args:
        df_filtered: DataFrame containing IV calculation results
        benchmark_col: Column name for the benchmark IV values
        output_file: Filename to save the plot
    """
    # Get all IV columns except benchmark and time columns
    iv_cols = [col for col in df_filtered.columns if col.startswith('iv_') and 
               col != benchmark_col and not col.endswith('_time')]
    
    # Create a filled version of the DataFrame
    df_filled = df_filtered.copy()
    for col in iv_cols + [benchmark_col]:
        df_filled[col].fillna(0, inplace=True)
    
    # Determine plot layout
    n_methods = len(iv_cols)
    n_cols = min(3, n_methods)
    n_rows = (n_methods + n_cols - 1) // n_cols
    
    # Create figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    if n_rows * n_cols > 1:
        axes = axes.flatten()
    else:
        axes = [axes]
    
    # Create scatter plots for each method
    for i, col in enumerate(iv_cols):
        if i < len(axes):
            # Get data
            x = df_filled[benchmark_col]
            y = df_filled[col]
            
            # Create scatter plot
            axes[i].scatter(x, y, alpha=0.7)
            
            # Add identity line
            min_val = min(x.min(), y.min())
            max_val = max(x.max(), y.max())
            axes[i].plot([min_val, max_val], [min_val, max_val], 'r--')
            
            # Add labels
            axes[i].set_xlabel(f'{benchmark_col} IV')
            axes[i].set_ylabel(f'{col} IV')
            
            # Calculate error metrics for title
            errors = y - x
            mae = np.abs(errors).mean()
            rmse = np.sqrt((errors**2).mean())
            
            # Set title
            method_name = col.replace('iv_', '').replace('_', ' ').title()
            axes[i].set_title(f'{method_name} vs Benchmark\nMAE: {mae:.4f}, RMSE: {rmse:.4f}')
            
            # Highlight NaN values with a different color
            nan_mask = df_filtered[col].isna()
            if nan_mask.any():
                axes[i].scatter(df_filled.loc[nan_mask, benchmark_col], 
                               df_filled.loc[nan_mask, col], 
                               color='red', alpha=0.3, 
                               label='NaN (set to 0)')
                axes[i].legend()
    
    # Remove unused subplots
    for i in range(n_methods, len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    
    return fig

def create_error_boxplot(df_filtered, benchmark_col='iv_benchmark', output_file='iv_error_boxplot.png'):
    """
    Create a boxplot of absolute errors for each method.
    
    Args:
        df_filtered: DataFrame containing IV calculation results
        benchmark_col: Column name for the benchmark IV values
        output_file: Filename to save the plot
    """
    # Get all IV columns except benchmark
    iv_cols = [col for col in df_filtered.columns if col.startswith('iv_') and 
               col != benchmark_col and not col.endswith('_time')]
    
    # Create a filled version of the DataFrame
    df_filled = df_filtered.copy()
    for col in iv_cols + [benchmark_col]:
        df_filled[col].fillna(0, inplace=True)
    
    # Create error DataFrame
    error_df = pd.DataFrame()
    for col in iv_cols:
        method_name = col.replace('iv_', '').replace('_', ' ').title()
        error_df[method_name] = np.abs(df_filled[col] - df_filled[benchmark_col])
    
    # Create boxplot
    plt.figure(figsize=(14, 8))
    sns.boxplot(data=error_df)
    plt.title('Absolute Errors Compared to Benchmark')
    plt.ylabel('Absolute Error')
    plt.xlabel('Method')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

def run_iv_error_analysis(df_filtered):
    """
    Run a complete error analysis on IV calculation methods.
    
    Args:
        df_filtered: DataFrame containing IV calculation results
    """
    # Make sure we have a benchmark column
    benchmark_col = 'iv_benchmark'
    if benchmark_col not in df_filtered.columns:
        raise ValueError(f"Benchmark column '{benchmark_col}' not found in DataFrame")
    
    # Calculate error metrics
    print("Calculating error metrics...")
    error_results = analyze_iv_results(df_filtered, benchmark_col)
    
    # Print the results
    print("\nError metrics for each IV method compared to benchmark:")
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', 150)  # Wider display
    print(error_results)
    
    # Save to CSV
    error_results.to_csv('iv_error_results.csv')
    
    # Print the best methods
    print("\nTop 3 methods by RMSE:")
    top_methods = error_results.head(3).index
    for i, method in enumerate(top_methods):
        print(f"{i+1}. {method}")
        print(f"   RMSE: {error_results.loc[method, 'RMSE']:.6f}")
        print(f"   Mean Absolute Error: {error_results.loc[method, 'Mean Absolute Error']:.6f}")
        print(f"   Mean % Error: {error_results.loc[method, 'Mean % Error']:.2f}%")
        print(f"   Valid %: {error_results.loc[method, 'Valid Percentage']:.1f}%")
    
    # Plot comparisons
    print("\nCreating comparison plots...")
    try:
        plot_iv_comparison(df_filtered, benchmark_col)
        print("Created scatter plot comparison: iv_comparison.png")
    except Exception as e:
        print(f"Error creating scatter plots: {e}")
    
    try:
        create_error_boxplot(df_filtered, benchmark_col)
        print("Created error boxplot: iv_error_boxplot.png")
    except Exception as e:
        print(f"Error creating boxplot: {e}")
    
    # Compare all to benchmark with filled NaNs
    df_results = df_filtered.copy()
    iv_cols = [col for col in df_results.columns if col.startswith('iv_') and not col.endswith('_time')]
    
    # Create dataframe for comparison
    comparison_df = pd.DataFrame()
    for col in iv_cols:
        comparison_df[col] = df_results[col]
    
    # Replace NaN with 0 and calculate differences
    comparison_df_filled = comparison_df.fillna(0)
    for col in [c for c in iv_cols if c != benchmark_col]:
        comparison_df_filled[f'{col}_diff'] = comparison_df_filled[col] - comparison_df_filled[benchmark_col]
    
    # Save detailed results
    comparison_df.to_csv('iv_values_comparison.csv')
    comparison_df_filled.to_csv('iv_values_comparison_filled.csv')
    
    # Print success rate for each method
    print("\nSuccess rate for each method:")
    for col in [c for c in iv_cols if c != benchmark_col]:
        success_rate = (df_filtered[col].notna().sum() / len(df_filtered)) * 100
        print(f"{col}: {success_rate:.1f}%")
    
    print("\nAnalysis complete! Results saved to CSV files.")
    return error_results



    

In [15]:
print(df_filtered.columns)

Index(['Unnamed: 0', 'minute_str', 'Instrument', 'mean_ask', 'mean_bid',
       'cumcashvol', 'cumvolume', 'openinterest', 'is_call', 'strike',
       'maturity', 'underly_mean_mid', 'mean_mid', 'twap', 'date_str',
       'time_to_expiry', 'current_underlying_mean_mid', 'iv_binomial_default',
       'iv_benchmark', 'iv_ql_binomial_default', 'iv_trinomial_default',
       'iv_baw_default', 'iv_bjerksund_default', 'iv_euro_default',
       'iv_monte_carlo_default', 'iv_zhu'],
      dtype='object')


In [16]:
print(df_filtered.columns)

Index(['Unnamed: 0', 'minute_str', 'Instrument', 'mean_ask', 'mean_bid',
       'cumcashvol', 'cumvolume', 'openinterest', 'is_call', 'strike',
       'maturity', 'underly_mean_mid', 'mean_mid', 'twap', 'date_str',
       'time_to_expiry', 'current_underlying_mean_mid', 'iv_binomial_default',
       'iv_benchmark', 'iv_ql_binomial_default', 'iv_trinomial_default',
       'iv_baw_default', 'iv_bjerksund_default', 'iv_euro_default',
       'iv_monte_carlo_default', 'iv_zhu'],
      dtype='object')


In [17]:
df_result=df_filtered[['iv_binomial_default', 'iv_benchmark',
       'iv_trinomial_default', 'iv_baw_default', 'iv_bjerksund_default', 'iv_euro_default', 'iv_monte_carlo_default', 'iv_zhu']]

In [18]:
error_results = run_iv_error_analysis(df_result)

Calculating error metrics...

Error metrics for each IV method compared to benchmark:
                        Total Count  Valid Count  Valid Percentage  Mean Absolute Error      RMSE  Mean % Error  Max Absolute Error
iv_trinomial_default           10.0         10.0             100.0             0.000152  0.000163      0.089925            0.000258
iv_binomial_default            10.0         10.0             100.0             0.000264  0.000284      0.162061            0.000387
iv_zhu                         10.0         10.0             100.0             0.000733  0.000959      0.462712            0.002177
iv_euro_default                10.0         10.0             100.0             0.001218  0.001463      0.762219            0.003085
iv_monte_carlo_default         10.0         10.0             100.0             0.004776  0.005391      2.792484            0.009388
iv_baw_default                 10.0         10.0             100.0             0.108168  0.108908     64.753666           